# **파이썬과 재무관련 실습**
**[Markdown Wiki](https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:TeX_%EB%AC%B8%EB%B2%95)** | **[Markdown Editor](http://www.hostmath.com/)**

```r
scipy => numpy 1.2 => numpy-financial
$ pip install numpy-financial
```
# **1 현금흐름, 이자율과 시간 가치 계산**
## **01 단리 / 복리 이자계산**
연 5% 이자를 제공하는 은행에 1천만원을 예금하고 1년후에 인출하기로 하였다. 1년후 받는 금액은?

In [1]:
cash_original = 1000
year = 1
rate = 0.05

# 단리계산 : 5% (연간)이자율 계산
cash_original * ((1 + rate) * year)

1050.0

In [2]:
# 복리계산 : 5% (연간)이자율을 월 단위 복리계산
years = 12
cash_original*(1+(rate/years))**years

1051.161897881733

In [3]:
# 복리계산 : 5% (연간)이자율을 주 단위 복리계산
years = 52
cash_original*(1+(rate/years))**years

1051.2458419272002

In [4]:
# 복리계산 : 5% (연간)이자율을 시간단위 복리계산
years = 8860
cash_original*(1+(rate/years))**years

1051.270948059031

# **2 NPV 와 IRR**
- **<span style="color:orange">미래의 이익을 예상하고 현재 부채를 발생</span>** 하는 경우, **<span style="color:orange">현재 시점에서 타당성을 판단</span>**

## **01 순현재가치 NPV (Net Present Value)**
- $ NPV = SUM(현금흐름 (1+할인율)^{i}) $
$$ NPV = \sum \frac{C (현금흐름) }{(1 + rate (할인율))^{i(기간)}} $$

In [5]:
# 70,000 을 대출하여 사업을 시작한 뒤, 연간 예상 수익을 활용한 가치계산
rate = 0.015
npv = -70000
cashflows = [ 12000 , 15000 , 18000 , 21000 , 26000 ]

for i, c in enumerate(cashflows):  
    npv = npv + c/(1+rate)**(i+1)
print(npv)

17516.929512135568


In [6]:
cashflows = [ -70000, 12000 , 15000 , 18000 , 21000 , 26000 ]
import numpy_financial as nf
nf.npv(rate, cashflows)

17516.929512135568

## **02 내부수익률 IRR (Internal Rate of Return)**
- 수익률 : <span style="color:orange">현재시점</span> 의 가치를 <span style="color:orange">미래 시점의 가치로 환산</span>
- 할인율 : 미래시점의 가치를 <span style="color:orange">현재 시점의 가치로 환산</span>
- IRR : 현금유입 과 현금유출 의 현재가치가 일치되면 NPV 가 0이 되고, 이때의 할인율 값
$$ IRR = NVP = \sum \frac{ C_{t} }{ (1 + rate)^{t} } - C_{0} = 0 $$

In [7]:
cashflows = [ -70000, 12000 , 15000 , 18000 , 21000 , 26000 ]

import numpy_financial as nf
irr = nf.irr(cashflows) # 내부수익률을 계산

# 구한 IRR을 npv의 할인율로 사용하여 NPV 확인 (정확하다면 NPV는 0)
npv = nf.npv(irr, cashflows)
f'NPV {npv:.2f} 일때 : IRR {irr:.1%}'

'NPV 0.00 일때 : IRR 8.7%'

# **3 수익률**
## **01 기간 수익률의 평균, 산술평균과 기하평균**
- 산술평균 : 합계를 갯수로 나눈다
$$ 산술평균 = \frac{1}{n}(r_{1} + r_{2} + ... + r_{n-1} + r_{n}) $$
- 기하평균 : 전체 곱을 갯수의 제곱근 나눈다
$$ 기하평균 = \sqrt[n]{ (1+r_{1}) + (1+r_{2}) + ... + (1+r_{n-1}) + (1+r_{n})-1)} $$
- 가중평균 : 사건별 중요도 및 영향 정도에 해당하는 가중치를 고려한 값

In [11]:
# 산술평균
returns = [ 0.1, 0.06, 0.05 ]
arithmetic_mean = sum(returns) /3
f'Arithmetic`s Mean is {arithmetic_mean:.2%}'

'Arithmetic`s Mean is 7.00%'

In [9]:
geometric_mean = 1
number_count = len(returns)

for _ in returns:
    geometric_mean = geometric_mean * (1 + _)
geometric_mean = geometric_mean ** (1 / number_count) - 1
f'Geometric Mean is {geometric_mean:.2%}'

'Geometric Mean is 6.98%'

In [10]:
# 모듈을 활용한 기하평균 계산
from statistics import geometric_mean
geo_mean = geometric_mean(returns)
f'Geometric Mean is {geo_mean:.2%}'

'Geometric Mean is 6.69%'

In [14]:
# 가중평균
score = [ 82, 90, 76]
weight = [0.2, 0.35, 0.45]
weighted_average = sum(s*w  for s, w in zip(score, weight))
f"Weighted Average Score is {weighted_average}"

'Weighted Average Score is 82.1'

## **02 종목 분산의 지배원리 (Dominance Principle)**
- 수익률 : 기대 수익률 활용
- 위험 : 수익률의 표준편차와 분산
- 둘을 종합하면, 두 포트폴리오 수익률의 표준편차가 작은 케이스
- 표준편차가 동일한 경우에는 기대수익률이 큰 포트폴리오를 선택

# **4 통계량**
## **01 평균과 기댓값**
- 기댓값 : 확률 (특정한 사건이 발생할 확률) 이 더해진 평균 값이다

$$ E(x) = \sum{p_{i}}{x_{i}} $$

In [19]:
case  = [1,2,3,4,5,6] 
prob = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]

expected_value = sum(c*p for c, p in zip(case, prob))
f"Expected Value is {expected_value}"

'Expected Value is 3.5'